# XML, HTML, and Web Scraping

JSON and XML are two different ways to represent hierarchical data. Which one is better? There are lots of articles online which discuss similarities and differences between JSON and XML and their advantages and disadvantages. Both formats are still in current usage, so it is good to be familiar with both. However, JSON is more common, so we'll focus on working with JSON representations of hierarchical data.

The reading covered an example of using Beautiful Soup to parse XML. Rather than doing another example XML now, we'll skip straight to scraping HTML from a webpage. Both HTML and XML can be parsed in a similar way with Beautiful Soup.

In [2]:
import pandas as pd

## Scraping an HTML table with Beautiful Soup

Open the URL https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population and scroll down until you see a table of the cities in the U.S. with population over 100,000 (as of Jul 1, 2022). We'll use Beautiful Soup to scrape information from this table.

Read in the HTML from the URL using the `requests` library.

In [3]:
# Reading in HTML

import requests

url = "https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population"

response = requests.get(url)

Use Beautiful Soup to parse this string into a tree called `soup`

In [4]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(response.content, "html.parser")

To find an HTML tag corresponding to a specific element on a webpage, right-click on it and choose "Inspect element". Go to the cities table Wikipedia page and do this now.

You should find that the cities table on the Wikipedia page corresponds to the element

```
<table class="wikitable sortable jquery-tablesorter" style="text-align:center">
```

There are many `<table>` tags on the page.

In [5]:
len(soup.find_all("table"))

14

We can use attributes like `class=` and `style=` to narrow down the list.

In [6]:
len(soup.find_all("table",
                  attrs={
                      "class": "wikitable sortable",
                      "style": "text-align:center"}
                  ))

3

At this point, you can manually inspect the tables on the webpage to find that the one we want is the first one (see `[0]` below). We'll store this as `table`.

In [7]:
table = soup.find_all("table",
                  attrs={
                      "class": "wikitable sortable",
                      "style": "text-align:center"}
                  )[0]

**Now you will write code to scrape the information in `table` to create a Pandas data frame with one row for each city and columns for: city, state, population (2022 estimate), and 2020 land area (sq mi).** Refer to the Notes/suggestions below as you write your code. A few Hints are provided further down, but try coding first before looking at the hints.

Notes/suggestions:

- Use as a guide the code from the reading that produced the data frame of Statistics faculty
- Inspect the page source as you write your code
- You will need to write a loop to get the information for all cities, but you might want to try just scraping the info for New York first
- You will need to pull the text from the tag. If `.text` returns text with "\n" at the end, try `.get_text(strip = True)` instead of `.text`
- Don't forget to convert to a Pandas Data Frame; it should have 333 rows and 4 columns
- The goal of this exercise is just to create the Data Frame. If you were going to use it --- e.g., what is the population density for all cities in CA? --- then you would need to clean the data first (to clean strings and convert to quantitative). (You can use Beautiful Soup to do some of the cleaning for you, but that goes beyond our scope.)

In [8]:
  # Initialize an empty list
rows = []

# Iterate over all rows in the cities table
for city in table.find_all("tr")[1:]:

    # Get all the cells (<td>) in the row.
    cells = city.find_all("td")

    # The information we need is the text between tags.

    # Find the the name of the city in cell[0]
    # which for most cities is contained in the <i> tag
    city_tag = cells[0].find("i") or cells[0]
    city = city_tag.get_text(strip = True)

    # Find the state of the city in cell[1]
    # which for most cities is contained in the <i> tag
    link = cells[1].find("i") or cells[1]
    state = link.get_text(strip = True)

    # Find the population of the city in cell[2]
    # which for most cities is contained in the <a> tag
    pop_tag = cells[2].find("a") or cells[2]
    pop = pop_tag.get_text(strip = True)

    # Find the land area of the city in cell[5]
    # which for most cities is contained in the <a> tag
    area_tag = cells[5].find("a") or cells[5]
    area = area_tag.get_text(strip = True)

    # Append this data.
    rows.append({
        "name": city,
        "state": state,
        "population": pop,
        "land_area":area
    })

In [19]:
# Converting Rows to Dataframe
cities_df = pd.DataFrame(rows)
cities_df

,name,state,population,land_area
0,New York,New York,"8,335,897",300.5 sq mi
1,Los Angeles,California,"3,822,238",469.5 sq mi
2,Chicago,Illinois,"2,665,039",227.7 sq mi
3,Houston,Texas,"2,302,878",640.4 sq mi
4,Phoenix,Arizona,"1,644,409",518.0 sq mi
...,...,...,...,...
328,Albany,New York,"100,826",21.4 sq mi
329,Hesperia,California,"100,744",72.7 sq mi
330,New Bedford,Massachusetts,"100,682",20.0 sq mi
331,Davenport,Iowa,"100,486",63.8 sq mi


Hints:

- Each city is a row in the table; find all the `<tr>` tags to find all the cities
- Look for the `<td>` tag to see table entries within a row
- The rank column is represented by `<th>` tags, rather than `<td>` tags. So within a row, the first (that is, `[0]`) `<td>` tag corresponds to the city name.

## Aside: Scraping an HTML table with Pandas



The Pandas command `read_html` can be used to scrape information from an HTML table on a webpage.

We can call `read_html` on the URL.

In [ ]:
pd.read_html("https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population")

However, this scrapes all the tables on the webpage, not just the one we want. As with Beautiful Soup, we can narrow the search by specifying the table attributes.

In [ ]:
pd.read_html("https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population", attrs = {'class': 'wikitable sortable', "style": "text-align:center"})

This still returns 3 tables. As we remarked above, the table that we want is the first one (see `[0]` below).

In [12]:
df_cities2 = pd.read_html("https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population", attrs = {'class': 'wikitable sortable', "style": "text-align:center"})[0]
df_cities2

,2022 rank,City,State[c],2022 estimate,2020 census,Change,2020 land area,2020 land area.1,2020 population density,2020 population density.1,Location
0,1,New York[d],New York,8335897,8804190,−5.32%,300.5 sq mi,778.3 km2,"29,298/sq mi","11,312/km2",".mw-parser-output .geo-default,.mw-parser-outp..."
1,2,Los Angeles,California,3822238,3898747,−1.96%,469.5 sq mi,"1,216.0 km2","8,304/sq mi","3,206/km2",34°01′N 118°25′W﻿ / ﻿34.02°N 118.41°W
2,3,Chicago,Illinois,2665039,2746388,−2.96%,227.7 sq mi,589.7 km2,"12,061/sq mi","4,657/km2",41°50′N 87°41′W﻿ / ﻿41.84°N 87.68°W
3,4,Houston,Texas,2302878,2304580,−0.07%,640.4 sq mi,"1,658.6 km2","3,599/sq mi","1,390/km2",29°47′N 95°23′W﻿ / ﻿29.79°N 95.39°W
4,5,Phoenix,Arizona,1644409,1608139,+2.26%,518.0 sq mi,"1,341.6 km2","3,105/sq mi","1,199/km2",33°34′N 112°05′W﻿ / ﻿33.57°N 112.09°W
...,...,...,...,...,...,...,...,...,...,...,...
328,329,Albany,New York,100826,99224,+1.61%,21.4 sq mi,55.4 km2,"4,637/sq mi","1,790/km2",42°40′N 73°48′W﻿ / ﻿42.67°N 73.80°W
329,330,Hesperia,California,100744,99818,+0.93%,72.7 sq mi,188.3 km2,"1,373/sq mi",530/km2,34°24′N 117°19′W﻿ / ﻿34.40°N 117.32°W
330,331,New Bedford,Massachusetts,100682,101079,−0.39%,20.0 sq mi,51.8 km2,"5,054/sq mi","1,951/km2",41°40′N 70°56′W﻿ / ﻿41.66°N 70.94°W
331,332,Davenport,Iowa,100486,101724,−1.22%,63.8 sq mi,165.2 km2,"1,594/sq mi",615/km2,41°34′N 90°36′W﻿ / ﻿41.56°N 90.60°W


Wait, that seemed much easier than using Beautiful Soup, and it returned a data frame, and we even got for free some formatting like removing the commas from the population! Why didn't we just use `read_html` in the first place? It's true the `read_html` works well when scraping information from an HTML *table*. Unfortunately, you often want to scrape information from a webpage that isn't conveniently stored in an HTML table, in which case `read_html` won't work. (It only searches for `<table>`, `<th>`, `<tr>`, and `<td>` tags, but there are many other HTML tags.) Though Beautiful Soup is not as simple as `read_html`, it is more flexible and thus more widely applicable.

## Scraping information that is NOT in a `<table>` with Beautiful Soup

The Cal Poly course catalog http://catalog.calpoly.edu/collegesandprograms/collegeofsciencemathematics/statistics/#courseinventory contains a list of courses offered by the Statistics department. **You will scrape this website to obtain a Pandas data frame with one row for each DATA or STAT course and two columns: course name and number (e.g, DATA 301. Introduction to Data Science) and term typically offered (e.g., Term Typically Offered: F, W, SP).**

Note: Pandas `read_html` is not help here since the courses are not stored in a `<table>.`

In [13]:
pd.read_html("http://catalog.calpoly.edu/collegesandprograms/collegeofsciencemathematics/statistics/#courseinventory")

[                                        Program name   Program type
 0                              Actuarial Preparation          Minor
 1  Cross Disciplinary Studies Minor in Bioinforma...          Minor
 2   Cross Disciplinary Studies Minor in Data Science          Minor
 3                                         Statistics  BS, MS, Minor]


Notes/suggestions:


- Inspect the page source as you write your code
- The courses are not stored in a `<table>`. How are they stored?
- You will need to write a loop to get the information for all courses, but you might want to try just scraping the info for DATA 100 first
- What kind of tag is the course name stored in? What is the `class` of the tag?
- What kind of tag is the quarter(s) the course is offered stored in? What is the `class` of the tag? Is this the only tag of this type with the class? How will you get the one you want?
- You don't have to remove the number of units (e.g., 4 units) from the course name and number, but you can try it if you want
- You will need to pull the text from the tag. If `.text` returns text with "\n" at the end, try `get_text(strip = True)` instead of `text`
- Don't forget to convert to a Pandas Data Frame; it should have 74 rows and 2 columns
- The goal of this exercise is just to create the Data Frame. If you were going to use it then you might need to clean the data first. (You can use Beautiful Soup to do some of the cleaning for you, but that goes beyond our scope.)



In [22]:
url_courses = "https://catalog.calpoly.edu/collegesandprograms/collegeofsciencemathematics/statistics/#courseinventory"


response_courses = requests.get(url_courses)

soup = BeautifulSoup(response_courses.content, "html.parser")

# Finding All Courses
courses = soup.find_all("div",
                        attrs=
                         {"class": "courseblock"})


In [27]:
# Iterate over all rows in the cities table
rows = []

for course in courses:
    cells = course.find_all("p")

    # Find the the name of the faculty in cell[0]
    # which for most courses is contained in the <strong> tag
    name_tag = cells[0].find("strong") or cells[0]
    name = name_tag.get_text(separator="\n", strip=True).split('.\n')[0]

    # Append this data.
    rows.append({
        "course_name": name,
    })

# Converting Rows to Dataframe
courses_df = pd.DataFrame(rows)
courses_df

,course_name
0,DATA 100. Data Science for All I
1,DATA 301. Introduction to Data Science
2,DATA 401. Data Science Process and Ethics
3,DATA 402. Mathematical Foundations of Data Sci...
4,DATA 403. Data Science Projects Laboratory
...,...
69,STAT 551. Statistical Learning with R
70,STAT 566. Graduate Consulting Practicum
71,STAT 570. Selected Advanced Topics
72,STAT 590. Graduate Seminar in Statistics


Hints:

- Each course is represented by a `<div>` with `class=courseblock`, so you can find all the courses with `soup.find_all("div", {"class": "courseblock"})`
- The course name is in a `<p>` tag with `class=courseblocktitle`, inside a `<strong>` tag. (Though I don't think we need to find the strong tag here.)
- The term typically offered is in `<p>` tag with `class=noindent`. However, there are several tags with this class; term typically offered is the first one.
- If you want to use Beautiful Soup to remove the course units (e.g., 4 units), find the `<span>` tag within the course name tag and `.extract()` this span tag